Install Environment Dependancies

In [ ]:
!pip install transformers torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 136.7 MB/s eta 0:00:01

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import os
import urllib.request
import zipfile
import pandas as pd

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU


Codebert-cpp is a finetuned model specficially for C++ language.  trained for 1,000,000 steps (with batch_size=32) on C++ code

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-cpp")
model = AutoModel.from_pretrained("neulab/codebert-cpp").to(device) #initialize model and rely on GPU
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.special_tokens_map)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-cpp and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


50265
512
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}


In [ ]:
bigvul_zip_url = "https://raw.githubusercontent.com/Meerschwein/Automating-SE/refs/heads/main/Big-Vul-dataset.zip"
data_path = "Big-Vul-dataset/data.json"

if not os.path.exists("Big-Vul-dataset.zip"):
    urllib.request.urlretrieve(bigvul_zip_url, "Big-Vul-dataset.zip")

if not os.path.exists("Big-Vul-dataset"):
    with zipfile.ZipFile("Big-Vul-dataset.zip", "r") as zip_ref:
        zip_ref.extractall("Big-Vul-dataset")

data_set = pd.read_json(data_path)
del data_set['bigvul_id']

In [ ]:

from sklearn.model_selection import train_test_split
train_data, test_eval_data = train_test_split(data_set, test_size=0.3, random_state=30)
test_data, eval_data = train_test_split(test_eval_data, test_size=0.33, random_state=30)




# 10% sample for training
sample_train,_ = train_test_split(train_data, test_size=0.9, random_state=30)


def tokenize_code(data):
    return tokenizer(data['code'], truncation=True, padding='max_length', max_length=512)


# 7:2:1 ratio
print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")
print(f"Testing set size: {len(eval_data)}")
print(f"Sample Training set size: {len(sample_train)}")
# tokenize each set
# tkn_train_data = train_data.apply(tokenize_code, axis=1)
# tkn_test_data = test_data.apply(tokenize_code, axis=1)
# tkn_eval_data = eval_data.apply(tokenize_code, axis=1)
tkn_train_sample = sample_train.apply(tokenize_code, axis=1)

# train_labels = torch.tensor(train_data['vul'].values, dtype=torch.float32).unsqueeze(1).to(device)
# test_labels = torch.tensor(test_data['vul'].values, dtype=torch.float32).unsqueeze(1).to(device)
# eval_labels = torch.tensor(eval_data['vul'].values, dtype=torch.float32).unsqueeze(1).to(device)
sample_labels = torch.tensor(sample_train['vul'].values, dtype=torch.float32).unsqueeze(1).to(device)
print(f"Finished Tokenizing")

Training set size: 130571
Testing set size: 37492
Testing set size: 18467
Sample Training set size: 13057
Finished Tokenizing


In [ ]:
from torch.utils.data import Dataset, DataLoader
#create dataset class inherting from torch dataset
class CodeVulnerabilityDataset(Dataset):
    def __init__(self, tokenized_data, labels):
      # tokenized_data is by defualt a dict to access index value, must be switched to list
        self.tokenized_data = list(tokenized_data)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        return {
            'input_ids': torch.tensor(item['input_ids'], dtype=torch.long).to(device),
            'attention_mask': torch.tensor(item['attention_mask'], dtype=torch.long).to(device),
            'labels': self.labels[idx]
        }

# Create the dataset
sample_dataset = CodeVulnerabilityDataset(tkn_train_sample, sample_labels)

# Create a DataLoader for shuffling and batching
batch_size = 32  #
sample_dataloader = DataLoader(sample_dataset, batch_size=batch_size, shuffle=True)


#sample one batch and check the device
for batch in sample_dataloader:
    print("Sample batch:")
    print(f"Input IDs shape: {batch['input_ids'].shape}, Device: {batch['input_ids'].device}")
    print(f"Attention Mask shape: {batch['attention_mask'].shape}, Device: {batch['attention_mask'].device}")
    print(f"Labels shape: {batch['labels'].shape}, Device: {batch['labels'].device}")
    break

Sample batch:
Input IDs shape: torch.Size([32, 512]), Device: cuda:0
Attention Mask shape: torch.Size([32, 512]), Device: cuda:0
Labels shape: torch.Size([32, 1]), Device: cuda:0


In [ ]:
from transformers import AutoModel
import torch.nn as nn

# Load the pre-trained CodeBERT base model
codebert_base = AutoModel.from_pretrained("neulab/codebert-cpp").to(device)

# Define a classification head with BiLSTM
class VulnerabilityClassifierWithLSTM(nn.Module):
    def __init__(self, base_model, hidden_size, num_layers, num_classes):
        super(VulnerabilityClassifierWithLSTM, self).__init__()
        self.base_model = base_model
        self.lstm = nn.LSTM(base_model.config.hidden_size, hidden_size, num_layers, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size * 2, num_classes) # *2 because of bidirectional

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        # Get the sequence of all hidden states
        sequence_output = outputs.last_hidden_state
        # Pass the sequence output through the LSTM
        lstm_output, _ = self.lstm(sequence_output)
        # Pool the LSTM output (e.g., take the mean over the sequence)
        pooled_output = torch.mean(lstm_output, dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Instantiate the classifier with BiLSTM
hidden_size = 64
num_layers = 2
num_classes = 1
model = VulnerabilityClassifierWithLSTM(codebert_base, hidden_size, num_layers, num_classes).to(device)

print("Vulnerability Classifier Model with BiLSTM:")
print(model)

Some weights of RobertaModel were not initialized from the model checkpoint at neulab/codebert-cpp and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Vulnerability Classifier Model with BiLSTM:
VulnerabilityClassifierWithLSTM(
  (base_model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_featur

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# # Separate vulnerable and non-vulnerable samples
# vulnerable_samples = tkn_train_sample[tkn_train_sample['vul'] == 1]
# non_vulnerable_samples = tkn_train_sample[tkn_train_sample['vul'] == 0]

# # Oversample the vulnerable samples to match the number of non-vulnerable samples
# oversampled_vulnerable = vulnerable_samples.sample(n=len(non_vulnerable_samples), replace=True, random_state=30)

# # Combine the oversampled vulnerable and original non-vulnerable samples
# balanced_sample_data = pd.concat([non_vulnerable_samples, oversampled_vulnerable])
# balanced_sample_data = balanced_sample_data.sample(frac=1, random_state=30).reset_index(drop=True) # Shuffle

# # Tokenize the balanced data
# tokenized_balanced_data = balanced_sample_data.apply(tokenize_code, axis=1)

# # Prepare labels for the balanced data
# balanced_sample_labels = torch.tensor(balanced_sample_data['vul'].values, dtype=torch.float32).unsqueeze(1).to(device)

# # Create the dataset and dataloader from the balanced data
# balanced_dataset = CodeVulnerabilityDataset(tokenized_balanced_data, balanced_sample_labels)
# balanced_dataloader = DataLoader(balanced_dataset, batch_size=16, shuffle=True)

# Re-train the model on the balanced data
import torch.optim as optim

# Reset the model (optional, but good practice when significantly changing data)
model = VulnerabilityClassifierWithLSTM(codebert_base, hidden_size, num_layers, num_classes).to(device)
criterion = nn.BCEWithLogitsLoss() # Revert to standard loss for now
optimizer = optim.AdamW(model.parameters(), lr=5e-2)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in sample_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(sample_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

print("Training finished on sample data!")

# Evaluate on the balanced data
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in sample_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities > 0.5).float()
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

print(f"\nEvaluation on the balanced (oversampled) dataset:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_labels, all_predictions)
print("\nConfusion Matrix:")
print(cm)


Confusion Matrix:
[[4731   34]
 [   0 4765]]


In [ ]:



vulnerable_samples = sample_data[sample_data['vul'] == 1]
for i in range(10):
# Tokenize and predict
    inputs_overflow = tokenizer(vulnerable_samples.iloc[i]['code'], return_tensors="pt", truncation=True, padding='max_length', max_length=512).to(device)
    model.eval()
    with torch.no_grad():
        outputs_overflow = model(**inputs_overflow)
        probability_overflow = torch.sigmoid(outputs_overflow).item()

    print(f"\nsample:\n{vulnerable_samples.iloc[i]['code']}")
    print(f"\nProbability of Vulnerability: {probability_overflow:.4f}")
    prediction_overflow = 1 if probability_overflow > 0.5 else 0
    print(f"Prediction (threshold={0.5}): {'Vulnerable' if prediction_overflow == 1 else 'Not Vulnerable'}")


sample:
void WebPluginProxy::InitiateHTTPRangeRequest(const char* url,
                                              const char* range_info,
                                              intptr_t existing_stream,
                                              bool notify_needed,
                                              intptr_t notify_data) {
  Send(new PluginHostMsg_InitiateHTTPRangeRequest(route_id_, url,
                                                  range_info, existing_stream,
                                                  notify_needed, notify_data));
 }


Probability of Vulnerability: 0.9989
Prediction (threshold=0.5): Vulnerable

sample:
void Chapters::Atom::ShallowCopy(Atom& rhs) const
{
    rhs.m_string_uid = m_string_uid;
    rhs.m_uid = m_uid;
    rhs.m_start_timecode = m_start_timecode;
    rhs.m_stop_timecode = m_stop_timecode;
    rhs.m_displays = m_displays;
    rhs.m_displays_size = m_displays_size;
    rhs.m_displays_count = m_displays_count;
}


Probabilit

In [ ]:
vulnerable_samples = sample_data[sample_data['vul'] == 0]
for i in range(10):
# Tokenize and predict
    inputs_overflow = tokenizer(vulnerable_samples.iloc[i]['code'], return_tensors="pt", truncation=True, padding='max_length', max_length=512).to(device)
    model.eval()
    with torch.no_grad():
        outputs_overflow = model(**inputs_overflow)
        probability_overflow = torch.sigmoid(outputs_overflow).item()

    print(f"\nsample:\n{vulnerable_samples.iloc[i]['code']}")
    print(f"\nProbability of Vulnerability: {probability_overflow:.4f}")
    prediction_overflow = 1 if probability_overflow > 0.5 else 0
    print(f"Prediction (threshold={0.5}): {'Vulnerable' if prediction_overflow == 1 else 'Not Vulnerable'}")


sample:
    virtual ~WorkerThreadTask() { }


Probability of Vulnerability: 0.0009
Prediction (threshold=0.5): Not Vulnerable

sample:
void DesktopWindowTreeHostX11::DispatchKeyEvent(ui::KeyEvent* event) {
  if (native_widget_delegate_->AsWidget()->IsActive())
    SendEventToSink(event);
}


Probability of Vulnerability: 0.0008
Prediction (threshold=0.5): Not Vulnerable

sample:
  AutocompleteHistoryManagerTest()
      : ui_thread_(ChromeThread::UI, &message_loop_) {
  }


Probability of Vulnerability: 0.0007
Prediction (threshold=0.5): Not Vulnerable

sample:
static void write_raw_data(bytearray_t * bplist, uint8_t mark, uint8_t * val, uint64_t size)
{
    uint8_t marker = mark | (size < 15 ? size : 0xf);
    byte_array_append(bplist, &marker, sizeof(uint8_t));
    if (size >= 15) {
        write_int(bplist, size);
    }
    if (BPLIST_UNICODE==mark) size <<= 1;
    byte_array_append(bplist, val, size);
}


Probability of Vulnerability: 0.0009
Prediction (threshold=0.5): Not Vulnerab